In [1]:
import pandas as pd

import sys
sys.path.insert(1, '../../scripts/')
from s3_support import *

- credit cards percentage
- alternate payment percentage
    - paypal/venmo
    - apple pay
    - echeck
    
channels:
0. standard
1. apple pay
2. google pay
3. venmo

In [42]:
q = '''select
            source,
            payment_type,
            channel,
            count(id) as transactions,
            sum(amount) as volume
        from transactions
        where 
            status='A' and
            date>=dateadd(month, -12, current_date) and
            source='p2p'
        group by payment_type, source, channel'''
df = redshift_query_read(q, schema='production')

In [58]:
_df = df.groupby(['payment_type', 'channel'])[['transactions', 'volume']].sum().reset_index()
_df['perc count'] = _df['transactions'] / _df['transactions'].sum()
_df['perc vol'] = _df['volume'] / _df['volume'].sum()
_df[['payment_type', 'perc count', 'perc vol']]

ccs = ['AM', 'DC', 'MC', 'VS']

ccs_standard = _df[(_df['payment_type'].isin(ccs))&(_df['channel']==0)][['perc count', 'perc vol']].sum()
ccs_standard['payment_type'] = "credit cards"

paypal_venmo = _df[(_df['channel']==3)|(_df['payment_type']=='pp')][['perc count', 'perc vol']].sum()
paypal_venmo['payment_type'] = "PayPal/Venmo"

echeck = _df[_df['payment_type']=='ck'][['perc count', 'perc vol']].sum()
echeck['payment_type'] = "ECheck"

apple_pay = _df[_df['channel']==1][['perc count', 'perc vol']].sum()
apple_pay['payment_type'] = "Apple Pay"

print("All sources:")
print("-"*40)
pd.DataFrame([ccs_standard, paypal_venmo, echeck, apple_pay])[['payment_type', 'perc count', 'perc vol']]

All sources:
----------------------------------------


,payment_type,perc count,perc vol
0,credit cards,0.862705,0.887969
1,PayPal/Venmo,0.001568,0.001149
2,ECheck,0.060735,0.066963
3,Apple Pay,0.028010,0.017241


In [57]:
_df = df[df['source']=='p2p'].groupby(['payment_type', 'channel'])[['transactions', 'volume']].sum().reset_index()
_df['perc count'] = _df['transactions'] / _df['transactions'].sum()
_df['perc vol'] = _df['volume'] / _df['volume'].sum()
_df[['payment_type', 'perc count', 'perc vol']]

ccs = ['AM', 'DC', 'MC', 'VS']

ccs_standard = _df[(_df['payment_type'].isin(ccs))&(_df['channel']==0)][['perc count', 'perc vol']].sum()
ccs_standard['payment_type'] = "credit cards"

paypal_venmo = _df[(_df['channel']==3)|(_df['payment_type']=='pp')][['perc count', 'perc vol']].sum()
paypal_venmo['payment_type'] = "PayPal/Venmo"

echeck = _df[_df['payment_type']=='ck'][['perc count', 'perc vol']].sum()
echeck['payment_type'] = "ECheck"

apple_pay = _df[_df['channel']==1][['perc count', 'perc vol']].sum()
apple_pay['payment_type'] = "Apple Pay"

print("P2P")
print("-"*40)
pd.DataFrame([ccs_standard, paypal_venmo, echeck, apple_pay])[['payment_type', 'perc count', 'perc vol']]

P2P
----------------------------------------


,payment_type,perc count,perc vol
0,credit cards,0.884552,0.890255
1,PayPal/Venmo,0.002836,0.001892
2,ECheck,0.010863,0.026584
3,Apple Pay,0.071796,0.056601


In [13]:
for source in df['source'].unique():
    print(source)
    print("-"*40)
    _df = df[df['source']==source].groupby('payment_type')[['transactions', 'volume']].sum().reset_index()
    _df['perc count'] = _df['transactions'] / _df['transactions'].sum()
    _df['perc vol'] = _df['volume'] / _df['volume'].sum()
    print(_df[['payment_type', 'perc count', 'perc vol']])
    print()

don_form
----------------------------------------
  payment_type  perc count  perc vol
0           AM    0.093811  0.174306
1           DC    0.042834  0.025304
2           MC    0.197527  0.189510
3           PP    0.052131  0.029338
4           VS    0.554520  0.502586
5           ck    0.059177  0.078955

mobile
----------------------------------------
  payment_type  perc count  perc vol
0           AM    0.075076  0.138642
1           DC    0.031588  0.021342
2           MC    0.203183  0.223437
3           PP    0.072044  0.040543
4           VS    0.580191  0.511542
5           ck    0.037917  0.064494

sms
----------------------------------------
  payment_type  perc count  perc vol
0           AM    0.066301  0.133993
1           BC    0.071865  0.006025
2           DC    0.020400  0.018936
3           MC    0.174793  0.163740
4           PP    0.036860  0.021724
5           VS    0.585388  0.586220
6           ck    0.044394  0.069362

vt
-------------------------------------

# alternative payment type distribution

In [34]:
q = '''select
            form,
            payment_type,
            channel,
            count(case when recurring=0 or recurring_origin=1 then id else null end) as trans_count,
            sum(case when recurring=0 or recurring_origin=1 then amount else null end) as trans_vol
        from transactions
        where
            status='A' and
            date>=dateadd(month, -12, current_date) and
            source='p2p' 
        group by form, channel, payment_type'''
trans = redshift_query_read(q, schema='production')

In [35]:
trans.tail(2)

,form,payment_type,channel,trans_count,trans_vol
11953,1018361,PP,0,1,50.0
11954,1021800,ck,0,1,2650.0


In [36]:
print("{:,} entries".format(len(trans)))
print("{:,} forms".format(len(trans['form'].unique())))

11,955 entries
2,690 forms


In [37]:
trans['payment_type'].unique()

array([None, 'VS', 'MC', 'PP', 'AM', 'DC', 'ck'], dtype=object)

In [38]:
# channel==0 standard
# channel==3 PayPal/Venmo
# channel==1 Apple Pay
# credit cards, echeck

data = []
ccs = ['MC', 'VS', 'AM', 'DC']

counter = 0
print("iterating through {:,} forms".format(len(trans['form'].unique())))
for form in trans['form'].unique():
    _df = trans[trans['form']==form]
    standard = _df[(_df['channel']==0)&(_df['payment_type'].isin(ccs))]
    data.append({
        'form': form,
        'standard CC count': standard['trans_count'].sum(),
        'standard CC vol': standard['trans_vol'].sum(),
        'PP/Venmo count': _df[_df['channel']==3]['trans_count'].sum(),
        'PP/Venmo vol': _df[_df['channel']==3]['trans_vol'].sum(),
        'Apple Pay count': _df[_df['channel']==1]['trans_count'].sum(),
        'Apple Pay vol': _df[_df['channel']==1]['trans_vol'].sum()
    })
    
    counter += 1
    if counter % 500 == 0:
        print("done with {:,} forms".format(counter))

iterating through 2,690 forms
done with 500 forms
done with 1,000 forms
done with 1,500 forms
done with 2,000 forms
done with 2,500 forms


In [39]:
df = pd.DataFrame(data)
df.head(2)

,form,standard CC count,standard CC vol,PP/Venmo count,PP/Venmo vol,Apple Pay count,Apple Pay vol
0,999716,659,85681.39,20,1340.75,0,0.00
1,1004614,386,37340.65,0,0.00,86,10354.35


In [40]:
df['all count'] = df[[c for c in df.columns if ' count' in c]].sum(axis=1)
df['all vol'] = df[[c for c in df.columns if ' vol' in c]].sum(axis=1)

df['standard count perc'] = df['standard CC count'] / df['all count']
df['PP/Venmo count perc'] = df['PP/Venmo count'] / df['all count']
df['Apple Pay count perc'] = df['Apple Pay count'] / df['all count']

df['standard vol perc'] = df['standard CC vol'] / df['all vol']
df['PP/Venmo vol perc'] = df['PP/Venmo vol'] / df['all vol']
df['Apple Pay vol perc'] = df['Apple Pay vol'] / df['all vol']

In [41]:
df[['standard count perc', 'PP/Venmo count perc', 'Apple Pay count perc']].agg(['mean', 'median']).reset_index()

,index,standard count perc,PP/Venmo count perc,Apple Pay count perc
0,mean,0.931573,0.004122,0.064306
1,median,1.000000,0.000000,0.000000


In [42]:
df[['standard vol perc', 'PP/Venmo vol perc', 'Apple Pay vol perc']].agg(['mean', 'median']).reset_index()

,index,standard vol perc,PP/Venmo vol perc,Apple Pay vol perc
0,mean,0.941472,0.002665,0.055863
1,median,1.000000,0.000000,0.000000
